論文  
https://arxiv.org/abs/2004.09484<br>
<br>
GitHub  
https://github.com/microsoft/Bringing-Old-Photos-Back-to-Life<br>
<br>
<a href="https://colab.research.google.com/github/kaz12tech/ai_demos/blob/master/Bringing_old_photos_back_to_life_demo.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 環境セットアップ

## GPU確認

In [ ]:
!nvidia-smi

## GitHubからコード取得

In [ ]:
!git clone https://github.com/microsoft/Bringing-Old-Photos-Back-to-Life.git photo_restoration

%cd /content/photo_restoration
# Commits on Jul 23, 2022使用
!git checkout 33875eccf4ebcd3665cf38cc56f3a0ce563d3a9c

## 依存するコードを取得

In [ ]:
# Synchronized-BatchNormをクローン
%cd /content/photo_restoration/Face_Enhancement/models/networks
!git clone https://github.com/vacancy/Synchronized-BatchNorm-PyTorch
!cp -rf Synchronized-BatchNorm-PyTorch/sync_batchnorm .

%cd /content/photo_restoration/Global/detection_models
!git clone https://github.com/vacancy/Synchronized-BatchNorm-PyTorch
!cp -rf Synchronized-BatchNorm-PyTorch/sync_batchnorm .

## ライブラリのインストール

In [ ]:
%cd /content/photo_restoration

!pip install -r requirements.txt

## ライブラリのインポート

In [ ]:
import os
import io
import IPython.display
import numpy as np
import PIL.Image

# 学習済みモデルのセットアップ

In [ ]:
# download the landmark detection model
%cd /content/photo_restoration/Face_Detection/
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!bzip2 -d shape_predictor_68_face_landmarks.dat.bz2

# download the pretrained model
%cd /content/photo_restoration/Face_Enhancement
!wget https://github.com/microsoft/Bringing-Old-Photos-Back-to-Life/releases/download/v1.0/face_checkpoints.zip
!unzip face_checkpoints.zip

%cd /content/photo_restoration/Global
!wget https://github.com/microsoft/Bringing-Old-Photos-Back-to-Life/releases/download/v1.0/global_checkpoints.zip
!unzip global_checkpoints.zip

# 描画用関数定義

In [ ]:
def imshow(a, format='png', jpeg_fallback=True):
  a = np.asarray(a, dtype=np.uint8)
  data = io.BytesIO()
  PIL.Image.fromarray(a).save(data, format)
  im_data = data.getvalue()
  try:
    disp = IPython.display.display(IPython.display.Image(im_data))
  except IOError:
    if jpeg_fallback and format != 'jpeg':
      print(('Warning: image was too large to display in format "{}"; '
             'trying jpeg instead.').format(format))
      return imshow(a, format='jpeg')
    else:
      raise
  return disp

def make_grid(I1, I2, resize=True):
  I1 = np.asarray(I1)
  H, W = I1.shape[0], I1.shape[1]
    
  if I1.ndim >= 3:
    I2 = np.asarray(I2.resize((W,H)))
    I_combine = np.zeros((H,W*2,3))
    I_combine[:,:W,:] = I1[:,:,:3]
    I_combine[:,W:,:] = I2[:,:,:3]
  else:
    I2 = np.asarray(I2.resize((W,H)).convert('L'))
    I_combine = np.zeros((H,W*2))
    I_combine[:,:W] = I1[:,:]
    I_combine[:,W:] = I2[:,:]
  I_combine = PIL.Image.fromarray(np.uint8(I_combine))

  W_base = 600
  if resize:
    ratio = W_base / (W*2)
    H_new = int(H * ratio)
    I_combine = I_combine.resize((W_base, H_new), PIL.Image.LANCZOS)

  return I_combine

# Old Photo Restoration

In [ ]:
%cd /content/photo_restoration/

input_folder = "test_images/old_w_scratch" # 入力画像ディレクトリ
output_folder = "test_output"    # 出力画像ディレクトリ

!mkdir {output_folder}

!python run.py \
  --input_folder {input_folder} \
  --output_folder {output_folder} \
  --GPU 0 \
  --with_scratch

In [ ]:
filenames = os.listdir(os.path.join(input_folder))
filenames.sort()

for filename in filenames:
  print(filename)
  image_original = PIL.Image.open(os.path.join(input_folder, filename))
  image_restore = PIL.Image.open(os.path.join(output_folder, 'final_output', filename))

  display(make_grid(image_original, image_restore))

## 任意の画像利用

In [ ]:
%cd /content/photo_restoration/

input_folder = "my_images/old_w_scratch" # 入力画像ディレクトリ
output_folder = "my_output"    # 出力画像ディレクトリ

!mkdir -p {input_folder}
!mkdir -p {output_folder}

In [ ]:
!wget -c https://i.pinimg.com/236x/ec/ac/65/ecac654443b3d6d93733538f4a30045b--old-portraits-vintage-photographs.jpg \
      -O my_images/old_w_scratch/test_01.png

!wget -c https://cdn.cambridgeincolour.com/images/tutorials/dpr_localized1.jpg \
      -O my_images/old_w_scratch/test_02.png

!wget -c https://www.phowd.com/images/landing/old-photo-restored-repaired-before.jpg \
      -O my_images/old_w_scratch/test_03.png

!apt -y install imagemagick
!convert -geometry "50%" my_images/old_w_scratch/test_03.png my_images/old_w_scratch/test_03.png

In [ ]:
!python run.py \
  --input_folder {input_folder} \
  --output_folder {output_folder} \
  --GPU 0 \
  --with_scratch

In [ ]:
filenames = os.listdir(os.path.join(input_folder))
filenames.sort()

for filename in filenames:
  print(filename)
  image_original = PIL.Image.open(os.path.join(input_folder, filename))
  image_restore = PIL.Image.open(os.path.join(output_folder, 'final_output', filename))

  display(make_grid(image_original, image_restore))